In [1]:
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
import os

from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain
from huggingface_hub import notebook_login
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain import HuggingFacePipeline
from langchain.text_splitter import CharacterTextSplitter
import textwrap
import sys
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory

import APIKEY

## RetrievalQA & Chat

In [2]:
#input_doc_pth = r'D:\nu_QA_data\m460bsp_Library_StdDriver_headers_1000'
#input_doc_pth = r'D:\nu_QA_data\m460bsp_StdDriver'
#input_doc_pth = r'D:\nu_QA_data\m2351bsp_1000'
#input_doc_pth = r'D:\nu_QA_data\m2351bsp_StdDriver_regs_1000'
#input_doc_pth = r'D:\nu_QA_data\m251bsp_StdDriver_1000'
input_doc_pth = r'D:\nu_QA_data\m251bsp_headers_1000'
# load embedding model
print("===== Load the embedding model =====")
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',model_kwargs={'device': 'cpu'})

# Create vectors store
print("===== Build FAISS =====")
#vectorstore=FAISS.from_documents(texts, embeddings)
vectorstore=FAISS.load_local(input_doc_pth, embeddings)

===== Load the embedding model =====
===== Build FAISS =====


In [3]:

os.environ["OPENAI_API_KEY"] = APIKEY.API_KEY_SERVICE_OPENAI
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

In [76]:
from langchain.chains.question_answering import load_qa_chain
from langchain import PromptTemplate 
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate

# Prompt
#template="""Use the following pieces of context to answer the question at the end.
#Please answer with C Code function as complete as possible.
#If you don't know the answer or the question has nothing to do with code or programing, don't try to make up an answer.
#{context}
#Question: {question}
#Answer:"""

#template="""Use the following pieces of context to answer the question at the end. The context is standard driver C header files of M251 MCU. 
#Please answer with C Code function as complete as possible.
#If you don't know the answer or the question has nothing to do with code or programing, don't try to make up an answer.
#{context}
#Question: {question}
#Answer:"""

#template="""Use the following pieces of context to answer the question at the end. The context is standard driver C header files of M251 MCU. 
#Please use context C functions as much as possible to answer with C code.
#If you don't know the answer or the question has nothing to do with code or programing, don't try to make up an answer.
#{context}
#Question: {question}
#Answer:"""



#template="""Use the following pieces of context and chat history to answer the question at the end. The context is standard driver C header files of M251 MCU. 
#Please answer with C Code function as complete as possible.
#If you don't know the answer or the question has nothing to do with code or programing, don't try to make up an answer.
#{context}
#{chat_history}
#Question: {question}
#Answer:"""
#
#QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question", "chat_history"], template=template)





# Define the system message template
system_template = """Use the following pieces of context and chat history to answer the question at the end. The context is standard driver C header files of M251 MCU. 
Please answer with C Code function as complete as possible.
If you don't know the answer or the question has nothing to do with code or programing, don't try to make up an answer.
----------------
{context}
{chat_history}"""

# Create the chat prompt templates
messages = [
SystemMessagePromptTemplate.from_template(system_template),
HumanMessagePromptTemplate.from_template("{question}")
]

qa_prompt = ChatPromptTemplate.from_messages(messages)


## RetrievalQA

In [82]:
retriever_vec=vectorstore.as_retriever(
    search_type="mmr", # Also test "similarity"
    search_kwargs={"k": 8})

# Normal memory
memory = ConversationBufferMemory(memory_key="chat_history", input_key='question', output_key='answer', return_messages=True)
#memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
# Should save the tokens
#memory = ConversationSummaryMemory(llm=llm, memory_key="chat_history", return_messages=True)

chain = ConversationalRetrievalChain.from_llm(llm, retriever=retriever_vec, memory=memory,
                                             return_source_documents=True, 
                                             combine_docs_chain_kwargs={"prompt": qa_prompt}
                                             )

chat_history = []

In [89]:
#query = "Generate a C source code which initializes I2C0 to access slave ROM with byte write and byte read operation, and check if the read data is equal to the write data."
#query = "Generate a C source code which TIMER0 MODE is TIMER_ONESHOT_MODE, TIMER0_FREQ is 1000000, TIMER0_PRESCALE_VALUE is 5, TIMER0_CMP_VALUE is 0x5A5A5A"
#query = "Generate a C source code which open EBI with bank2, 16BIT width, and fast timing"
#query = "Generate a C source code to get data from UART, PDMA to memory address 0x20000000, and do CRC32 at 0x20000000"
#query = "Write a C code of ACMP comparing DAC output with ACMP1_P1"
#query = "Write a C code to set the BPWM0 channel 0 for capture function"
#query = "Write a C code to set the BPWM0 channel 0 to output waveform with frequency 240000Hz and duty 50%"

#query = "Hi my name is Gary"
#query = "Whats my name"
query = "If I want to use multibytes write and multibytes read, how should I update this code?"

#result = chain({"question": query, "chat_history": chat_history})
result = chain({"question": query})
chat_history.extend([(query, result["answer"])])
#result = chain("Write a C code of ACMP comparing DAC output with ACMP1_P1")


In [74]:
for i in range(len(result['source_documents'])):
    print(result['source_documents'][i].metadata)

{'source': 'm251bsp_StdDriver\\StdDriver\\inc\\i2c.h', 'language': <Language.C: 'c'>}
{'source': 'm251bsp_StdDriver\\StdDriver\\inc\\i2c.h', 'language': <Language.C: 'c'>}
{'source': 'm251bsp_StdDriver\\StdDriver\\inc\\spi.h', 'language': <Language.C: 'c'>}
{'source': 'm251bsp_StdDriver\\StdDriver\\inc\\dac.h', 'language': <Language.C: 'c'>}
{'source': 'm251bsp_StdDriver\\StdDriver\\inc\\i2c.h', 'language': <Language.C: 'c'>}
{'source': 'm251bsp_StdDriver\\StdDriver\\inc\\i2c.h', 'language': <Language.C: 'c'>}
{'source': 'm251bsp_StdDriver\\StdDriver\\inc\\i2c.h', 'language': <Language.C: 'c'>}
{'source': 'm251bsp_StdDriver\\StdDriver\\inc\\i2c.h', 'language': <Language.C: 'c'>}


In [88]:
print(result['answer'])

Sure! Here's an example of C source code that initializes I2C0 to access a slave ROM with byte write and byte read operations, and checks if the read data is equal to the write data:

```c
#include "M251.h"

#define I2C_PORT I2C0
#define SLAVE_ROM_ADDR 0x50

void I2C_Init(void)
{
    // Enable I2C0 clock
    CLK_EnableModuleClock(I2C0_MODULE);

    // Set I2C0 multi-function pins
    SYS->GPA_MFPL &= ~(SYS_GPA_MFPL_PA9MFP_Msk | SYS_GPA_MFPL_PA8MFP_Msk);
    SYS->GPA_MFPL |= (SYS_GPA_MFPL_PA9MFP_I2C0_SCL | SYS_GPA_MFPL_PA8MFP_I2C0_SDA);

    // Configure I2C0 as master, 100kHz
    I2C_Open(I2C_PORT, 100000);

    // Enable I2C0 interrupt
    I2C_EnableInt(I2C_PORT);
    NVIC_EnableIRQ(I2C0_IRQn);
}

void I2C_WriteByte(uint8_t data)
{
    // Write data to I2C data register
    I2C_SET_DATA(I2C_PORT, data);

    // Send start condition, write address, and data
    I2C_START(I2C_PORT);
    I2C_WAIT_READY(I2C_PORT);
    I2C_SET_DATA(I2C_PORT, SLAVE_ROM_ADDR << 1);
    I2C_SET_CONTROL_REG(I2

In [90]:
print(result['answer'])

To update the code to use multibyte write and multibyte read operations, you can modify the `I2C_WriteByte` and `I2C_ReadByte` functions to use the `I2C_WriteMultiBytes` and `I2C_ReadMultiBytes` functions respectively. Here's the updated code:

```c
#include "M251.h"

#define I2C_PORT I2C0
#define SLAVE_ROM_ADDR 0x50

void I2C_Init(void)
{
    // Enable I2C0 clock
    CLK_EnableModuleClock(I2C0_MODULE);

    // Set I2C0 multi-function pins
    SYS->GPA_MFPL &= ~(SYS_GPA_MFPL_PA9MFP_Msk | SYS_GPA_MFPL_PA8MFP_Msk);
    SYS->GPA_MFPL |= (SYS_GPA_MFPL_PA9MFP_I2C0_SCL | SYS_GPA_MFPL_PA8MFP_I2C0_SDA);

    // Configure I2C0 as master, 100kHz
    I2C_Open(I2C_PORT, 100000);

    // Enable I2C0 interrupt
    I2C_EnableInt(I2C_PORT);
    NVIC_EnableIRQ(I2C0_IRQn);
}

void I2C_WriteMultiBytes(uint8_t data[], uint32_t length)
{
    // Send start condition, write address, and data
    I2C_START(I2C_PORT);
    I2C_WAIT_READY(I2C_PORT);
    I2C_SET_DATA(I2C_PORT, SLAVE_ROM_ADDR << 1);
    I2C_SET_CO

In [58]:
chain.memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='Hi my name is Gary'), AIMessage(content="Sorry, but I can't provide the answer you're looking for.")]), output_key='answer', input_key='question', return_messages=True, memory_key='chat_history')

In [49]:
chat_history

[('Hi my name is Gary', 'Hello Gary! How can I assist you today?'),
 ('What is my name', "I am an AI assistant and I don't have a name."),
 ('Whats my name',
  "I'm sorry, but I am an AI language model and I do not have the capability to know your name.")]

## Another choose load_qa_chain()

In [5]:

# Docs Retriever
#retriever=vectorstore.as_retriever(search_type="mmr", # Also test "similarity"
#                                   search_kwargs={"k": 8})
retriever=vectorstore.as_retriever(search_type="similarity", # Also test "similarity"
                                   search_kwargs={"k": 8})

# question & vector search
question = "Write a C code of ACMP comparing DAC output with ACMP1_P1"
docs = retriever.get_relevant_documents(question)

# Chain
chain = load_qa_chain(llm, chain_type="stuff", prompt=QA_CHAIN_PROMPT) 

In [ ]:
result = chain({"input_documents": docs, "question": question}, return_only_outputs=True)

In [ ]:
print(len(docs))
print(result['output_text'])

## ConversationSummaryMemory

In [ ]:
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

memory = ConversationSummaryMemory(llm=llm,memory_key="chat_history",return_messages=True)
qa = ConversationalRetrievalChain.from_llm(llm, retriever=vectorstore.as_retriever(), memory=memory)

In [ ]:
#query = "Write a C code of ACMP comparing DAC output with ACMP1_P1"
#query = "Write a C code of BMC data transfer with PDMA"
#query = "Write a C code function that reverse a string, input a string in parameter and return the reversed string"
query = "Give me a C code of configure BMC example"
result = qa(query)

In [ ]:
print(result['answer'])